# 🌍 Complete LULC Prediction System for Kovilpatti

## Causal Spatiotemporal Transformer for Land Use/Land Cover Prediction

This notebook provides a complete interactive workflow for LULC prediction.

## 1. Installation and Imports

In [ ]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent))

import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm

from src.data_generator import KovilpattiLULCGenerator
from src.dataset import TemporalLULCDataset
from src.model import CausalSpatiotemporalTransformer, PhysicsInformedLoss
from src.train import train_epoch, validate
from src.utils import set_seed, predict_future_multistep, create_confusion_matrix

print('✅ Imports successful!')
print(f'PyTorch version: {torch.__version__}')
print(f'CUDA available: {torch.cuda.is_available()}')

## 2. Configuration

In [ ]:
set_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

config = {
    'data': {'img_size': 256, 'num_classes': 7, 'train_samples': 200, 'val_samples': 40, 'test_samples': 40},
    'model': {'d_model': 256, 'n_heads': 8, 'n_layers': 4, 'dropout': 0.1},
    'training': {'batch_size': 8, 'num_epochs': 50, 'learning_rate': 0.0001, 'lambda_physics': 0.1, 'lambda_continuity': 0.05}
}

## 3. Data Generation

In [ ]:
generator = KovilpattiLULCGenerator(img_size=256, num_classes=7)
sequence = generator.generate_temporal_sequence(num_timesteps=5)

fig, axes = plt.subplots(1, 5, figsize=(20, 4))
for idx, lulc_map in enumerate(sequence):
    rgb = generator.lulc_to_rgb(lulc_map)
    axes[idx].imshow(rgb)
    axes[idx].set_title(f'Year {2018+idx}')
    axes[idx].axis('off')
plt.tight_layout()
plt.show()

## 4. Generate Full Dataset

In [ ]:
data_dir = Path('../data/Kovilpatti_LULC')
for split, n in [('train', 200), ('val', 40), ('test', 40)]:
    split_dir = data_dir / split
    split_dir.mkdir(parents=True, exist_ok=True)
    for i in tqdm(range(n), desc=split):
        seq = generator.generate_temporal_sequence(5)
        np.save(split_dir / f'sequence_{i:04d}.npy', np.stack(seq))
print('✅ Dataset generated!')

## 5. Load Datasets and Create Dataloaders

In [ ]:
train_dataset = TemporalLULCDataset(str(data_dir), 'train', 256, 7)
val_dataset = TemporalLULCDataset(str(data_dir), 'val', 256, 7)
test_dataset = TemporalLULCDataset(str(data_dir), 'test', 256, 7)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=2)

print(f'Train: {len(train_dataset)}, Val: {len(val_dataset)}, Test: {len(test_dataset)}')

## 6. Initialize Model

In [ ]:
model = CausalSpatiotemporalTransformer(num_classes=7, d_model=256, n_heads=8, n_layers=4).to(device)
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Model parameters: {num_params:,}')

criterion = torch.nn.CrossEntropyLoss()
physics_loss_fn = PhysicsInformedLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0001, weight_decay=0.0001)

## 7. Training Loop

In [ ]:
history = {'train_loss': [], 'val_loss': [], 'train_accuracy': [], 'val_accuracy': []}
training_config = {'lambda_physics': 0.1, 'lambda_continuity': 0.05}

for epoch in range(10):  # Reduced for demo
    train_metrics = train_epoch(model, train_loader, criterion, physics_loss_fn, optimizer, device, training_config)
    val_metrics = validate(model, val_loader, criterion, physics_loss_fn, device, training_config)
    
    history['train_loss'].append(train_metrics['loss'])
    history['val_loss'].append(val_metrics['loss'])
    history['train_accuracy'].append(train_metrics['accuracy'])
    history['val_accuracy'].append(val_metrics['accuracy'])
    
    print(f"Epoch {epoch+1}: Train Loss={train_metrics['loss']:.4f}, Val Loss={val_metrics['loss']:.4f}")

print('✅ Training complete!')

## 8. Visualization

In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history['train_loss'], label='Train')
plt.plot(history['val_loss'], label='Val')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss Curves')

plt.subplot(1, 2, 2)
plt.plot(history['train_accuracy'], label='Train')
plt.plot(history['val_accuracy'], label='Val')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Accuracy Curves')
plt.tight_layout()
plt.show()

## 9. Test Evaluation and Predictions

In [ ]:
test_metrics = validate(model, test_loader, criterion, physics_loss_fn, device, training_config)
print(f"Test Accuracy: {test_metrics['accuracy']:.4f}")
print(f"Test F1: {test_metrics['f1']:.4f}")
print(f"Test Kappa: {test_metrics['kappa']:.4f}")

## 10. Multi-Step Forecasting

In [ ]:
inputs, target, _ = next(iter(test_loader))
sample_input = inputs[0]  # First sample

future_preds = predict_future_multistep(model, sample_input, num_steps=3, device=device)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))
for idx, pred in enumerate(future_preds):
    pred_class = torch.argmax(pred, dim=0).numpy()
    rgb = generator.lulc_to_rgb(pred_class)
    axes[idx].imshow(rgb)
    axes[idx].set_title(f'Year {2023+idx}')
    axes[idx].axis('off')
plt.tight_layout()
plt.show()

## 11. Summary

This notebook demonstrated:
- Data generation for Kovilpatti LULC
- Model training with physics constraints
- Prediction and evaluation
- Multi-step forecasting